In [ ]:
from email import message
from dotenv import load_dotenv
from dotenv.main import logger
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

load_dotenv(override=True)
def push(text):
    print("push " + text + os.getenv("PUSHOVER_TOKEN") + " " + os.getenv("PUSHOVER_USER") )
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
        
    )

def show(text):
    try:
        #Console().print(text)
        logger.info(text)
    except Exception:
        logger.error(text)

@function_tool
def record_user_details(email: str, name: str = "Name not provided", notes: str = "not provided"):
    push(f"Recording {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}
    

@function_tool
def record_unknown_question(question: str):
    push(f"Recording {question}")
    return {"recorded": "ok"}

class Me:

    def __init__(self):
        self.openai = OpenAI()
        self.name = "Ed Donner"
        self.history=[]
        reader = PdfReader("me/linkedin.pdf")
        self.linkedin = ""
        self.tools = [record_user_details, record_unknown_question]
        for page in reader.pages:
            text = page.extract_text()
            if text:
                self.linkedin += text
        with open("me/summary.txt", "r", encoding="utf-8") as f:
            self.summary = f.read()
        
        def create_agent(self):
            return Agent(
                name="Professional Career Agent",
                instructions=self.system_prompt(),
                tools = self.tools,
                model="gpt-4o-mini"
           
        )
        
        self.agent = create_agent(self)

        
      
    def system_prompt(self):
        system_prompt = f"You are acting as {self.name}. You are answering questions on {self.name}'s website, \
particularly questions related to {self.name}'s career, background, skills and experience. \
Your responsibility is to represent {self.name} for interactions on the website as faithfully as possible. \
You are given a summary of {self.name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

        system_prompt += f"\n\n## Summary:\n{self.summary}\n\n## LinkedIn Profile:\n{self.linkedin}\n\n"
        system_prompt += f"With this context, please chat with the user, always staying in character as {self.name}."
        return system_prompt
    
  

    

    
       
    def conversation_history(self,message, history):
        messages = []
        
        # Add conversation history
        for msg in history:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    messages.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })
        
        # Add current user message
        messages.append({"role": "user", "content": message})
        print(messages)
        return messages
             
    async def chat(self, message, history):
        messages = self.conversation_history(message, history)
        
        # Run agent with full conversation history
        with trace("Automated SDR"):
            result = await Runner.run(self.agent, messages)
            show(result.final_output)
            return result.final_output

In [45]:
# Initialize and test the chat function
# In Jupyter notebooks, use 'await' directly instead of asyncio.run()
me = Me()
result = await me.chat("what is your name", [])
print(result)

My name is Ed Donner. How can I assist you today?


In [46]:
if __name__ == "__main__":
    me = Me()
    gr.ChatInterface(me.chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
